In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
from pprint import pprint
from datetime import datetime, timedelta, timezone
import pytz
import scipy.stats
from dateutil.relativedelta import relativedelta

import warnings

# Suppress the specific warnings
warnings.filterwarnings("ignore")

today = datetime.utcnow()

from api_keys import espn_link

# Pull in the data files from scrape

In [3]:
homeOnlySked = pd.read_csv('data/sked.csv', index_col=False)
homeOnlySked

completeSked = pd.read_csv('data/sked_full.csv', index_col=False)
completeSked

team_names = pd.read_csv('data/team_names.csv', index_col=False)
team_names

file_name = f"data/allG_df_raw-{today.strftime('%Y-%m-%d')}.csv"
allG_df = pd.read_csv(file_name, index_col=False)
allG_df

file_name = f"data/all_df_raw-{today.strftime('%Y-%m-%d')}.csv"
all_df = pd.read_csv(file_name, index_col=False)
all_df

team_names_only = team_names[['abbreviation', 'name']]

all_df_forwards =  all_df.loc[all_df['position'] != 'D']
all_df_defense =  all_df.loc[all_df['position'] == 'D']

all_df_centers =  all_df.loc[all_df['secondaryPosition'] == 'C']
all_df_wingers =  all_df.loc[all_df['secondaryPosition'] == 'W']

file_name = f"data/summary_stats-{today.strftime('%Y-%m-%d')}.csv"
player_bios = pd.read_csv(file_name, index_col=False)

file_name = f"data/rankings_worksheet_{(today-timedelta(days=5)).strftime('%Y-%m-%d')}.xlsx"
rankings = pd.read_excel(file_name)

FileNotFoundError: [Errno 2] No such file or directory: 'data/rankings_worksheet_2023-11-14.xlsx'

In [ ]:
ordercolumns = rankings.columns.to_list()
ordercolumns

In [ ]:
column_order = ['espnId',
 'FullName',
 'fantasyRemain',
 'rank_2023-11-20',
 'rank_2023-11-13',
 'rank_2023-11-06',
 'rank_2023-10-30',
 'rank_2023-10-23',
 'rank_2023-09-01'
]

In [ ]:
for index, row in rankings.iterrows():
    try:
        rankings.at[index,'fantasyRemain'] = player_bios.loc[player_bios['espnId'] == row['espnId']]['fantasyPointsRemain'].iloc[0]
    except:
        rankings.at[index,'fantasyRemain'] = 0


rankings['rank_2023-11-20'] = rankings['fantasyRemain'].rank(ascending=False, method='min').astype(int)
rankings 

rankings = rankings[column_order]
rankings.sort_values('rank_2023-11-20')

fixes = {
    'Tim Stutzle': 'Tim Stützle',
    'Jani Hakanpaa': 'Jani Hakanpää',
    'Benoit-Olivier Groulx': 'Bo Groulx',
    'Jesse Ylonen': 'Jesse Ylönen',
    'Alexis Lafreniere': 'Alexis Lafrenière',
    'Gustav Lindstrom': 'Gustav Lindström',
    'Alexander Kerfoot': 'Alex Kerfoot',
#     'Johnny Beecher': 'John Beecher',
    'Samuel Walker': 'Sammy Walker',
    'Alex Barre-Boulet': 'Alex Barré-Boulet'
}

inverse_fixes = {value: key for key, value in fixes.items()}

rankings['FullName'].replace(fixes, inplace=True)
rankings

In [ ]:
player_bios
remain_rank = 0
for index, row in player_bios.iterrows():
    try:
        rank = rankings.loc[rankings['espnId'] == row['espnId']]['rank_2023-11-20'].iloc[0]
        last_rank = rankings.loc[rankings['espnId'] == row['espnId']]['rank_2023-11-13'].iloc[0]
    except:
        rank = 300
        last_rank = 300
        
    remain_rank += 1
        
    player_bios.at[index, 'remain_rank'] =  remain_rank
    player_bios.at[index, 'last_rank'] =  last_rank
    player_bios.at[index, 'rank'] =  rank
    
not_ranked = player_bios.loc[player_bios['rank'] == 300]
for index, row in not_ranked.iterrows():
    print(f"{row['name']}: {row['remain_rank']}")
    if row['remain_rank'] > 300:
        break

print('-------------')        

poor_rank = player_bios.loc[(player_bios['rank'] >= 250) & (player_bios['last_rank'] <= 250)]
for index, row in poor_rank.iterrows():
    print(f"{row['name']}: {row['remain_rank']}")


In [ ]:
def convert_to_slug(name):
    # Convert to lowercase and replace spaces with hyphens
    slug = name.lower().replace(' ', '-')
    return slug

In [ ]:
player_bios['rank_by_pos'] = player_bios.groupby('default_pos')['rank'].rank(ascending=True)
player_bios.head(50)

In [ ]:
# player_bios.to_excel('data/manual-adjustments.xlsx')

player_bios_adjusted = pd.read_excel('data/manual-adjustments.xlsx')

# vasil = {'playerId': 8476883, 'name': 'A. Vasilevskiy', 'team': 'TBL', 'position': 'G', 'default_pos': 'G', 'espnId': 2976847, 'remain_rank': 55, 'fullName': 'Andrei Vasilevskiy'}
# player_bios_adjusted = player_bios_adjusted.append(vasil, ignore_index=True)


player_bios_adjusted = player_bios_adjusted.sort_values('remain_rank', ascending=True)
player_bios_adjusted['rank'] = player_bios_adjusted['remain_rank'].rank(ascending=True, method='first')
player_bios_adjusted['rank_by_pos'] = player_bios_adjusted.groupby('default_pos')['rank'].rank(ascending=True)
player_bios_adjusted.head(60)

In [ ]:
espn_teams = {
    'BOS': 'Bos',
    'VGK': 'Vgk',
    'NYR': 'NYR',
    'VAN': 'Van',
    'LAK': 'LA',
    'DAL': 'Dal',
    'FLA': 'Fla',
    'COL': 'Col',
    'WPG': 'Wpg',
    'TOR': 'Tor',
    'WSH': 'Wsh',
    'CAR': 'Car',
    'TBL': 'TB',
    'PHI': 'Phi',
    'DET': 'Det',
    'ANA': 'Ana',
    'ARI': 'Ari',
    'SEA': 'Sea',
    'NJD': 'NJ',
    'STL': 'StL',
    'NYI': 'NYI',
    'OTT': 'Ott',
    'PIT': 'Pit',
    'MTL': 'Mon',
    'BUF': 'Buf',
    'CGY': 'Cgy',
    'MIN': 'Min',
    'NSH': 'Nsh',
    'CBJ': 'Cls',
    'EDM': 'Edm',
    'CHI': 'Chi',
    'SJS': 'SJ'
}

team_names_only['espn_name'] = team_names_only['abbreviation'].map(espn_teams)
team_names_only

In [ ]:
player_bios_adjusted['fullName'].replace(inverse_fixes, inplace=True)

player_bios_adjusted

In [ ]:
# 1. <a href="http://www.espn.com/nhl/player/_/id/4024123/auston-matthews">Auston Matthews</a>, C, Tor (C1) <br />

ranking_module = pd.DataFrame()

for index, row in player_bios_adjusted.iterrows():
    base_string = '<a href="http://www.espn.com/nhl/player/_/id/'
    rank = str(int(row['rank']))
    Pid = str(row['espnId']) + '/'
    name = row['fullName']
    slug = convert_to_slug(name)
    pos = row['default_pos']
    team = team_names_only.loc[team_names_only['abbreviation'] == row['team']]['espn_name'].iloc[0]
    pos_rank = pos + str(int(row['rank_by_pos']))
    
    assemble = rank + '. ' + base_string + Pid + slug + '">' + name + '</a>, ' + pos + ', ' + team + ' (' + pos_rank + ')'
    
    temp = pd.DataFrame({'column': [assemble]})
    
    ranking_module = pd.concat([ranking_module, temp])
    
    print(f"{assemble}")
    
    if int(rank) == 250:
        break


In [ ]:
fileName = f"data/2024Rankings-05-{(today+timedelta(days=1)).strftime('%b%d')}.xlsx"
ranking_module.to_excel(fileName, index=False, header=False)

In [ ]:
player_bios